In [1]:
from agential.cog.agent.critic import CriticAgent
from langchain_community.tools.google_serper.tool import GoogleSerperRun
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    # Prompts.
    CRITIC_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC
)


import warnings
warnings.filterwarnings("ignore")

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

In [3]:
agent = CriticAgent(llm=llm, mode="search", search=search)

In [ ]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'
out = agent.generate(
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,  # HOTPOTQA_FEWSHOT_EXAMPLES_REACT
    prompt=CRITIC_INSTRUCTION_HOTPOTQA,
    critique_examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA
)